In [1]:
! pip install -q tensorflow-gpu==2.0.0.alpha0
import tensorflow as tf
print(tf.__version__)

    100% |████████████████████████████████| 332.1MB 51kB/s 
    100% |████████████████████████████████| 3.0MB 8.0MB/s 
    100% |████████████████████████████████| 419kB 11.6MB/s 
    100% |████████████████████████████████| 61kB 25.3MB/s 
2.0.0-alpha0


In [0]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt

In [0]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [10]:
print(x_train.shape)

(60000, 28, 28)


In [0]:
x_train = np.reshape(x_train, (len(x_train), 28,28,1))
x_test = np.reshape(x_test, (len(x_test), 28,28,1))

11493376/11490434 [==============================] - 0s 0us/step


In [24]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_train.shape

(60000, 784)

In [0]:
class Encoder(tf.keras.layers.Layer):
    '''Encodes a digit from the MNIST dataset'''
    
    def __init__(self,
                n_dims,
                name='encoder',
                **kwargs):
        super(Encoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = len(n_dims)
        self.encode_layer = layers.Dense(self.n_dims, activation='relu')
        
    @tf.function        
    def call(self, inputs):
        return self.encode_layer(inputs)
        
class Decoder(tf.keras.layers.Layer):
    '''Decodes a digit from the MNIST dataset'''

    def __init__(self,
                n_dims,
                name='decoder',
                **kwargs):
        super(Decoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = len(n_dims)
        self.decode_middle = layers.Dense(n_dims[0], activation='relu')
        self.recon_layer = layers.Dense(n_dims[1], activation='sigmoid')
        
    @tf.function        
    def call(self, inputs):
        x = self.decode_middle(inputs)
        return self.recon_layer(x)
        
        
    
class Autoencoder(tf.keras.Model):
    '''Vanilla Autoencoder for MNIST digits'''
    
    def __init__(self,
                 n_dims=[200, 392, 784],
                 name='autoencoder',
                 **kwargs):
        super(Autoencoder, self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.encoder = Encoder(n_dims[0])
        self.decoder = Decoder([n_dims[0], n_dims[1]])
        
    @tf.function        
    def call(self, inputs):
        x = self.encoder(inputs)
        

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
import tensorflow.keras.layers as layers

# Remove this comment from autoencoder_models.Autoencoder import Autoencoder

mnist = tf.keras.datasets.mnist


def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test


def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]


(X_train, _), (X_test, _) = mnist.load_data()
X_train = tf.cast(np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2])), tf.float64)
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2])), tf.float64)

X_train, X_test = standard_scale(X_train, X_test)

train_data = tf.data.Dataset.from_tensor_slices(X_train).batch(128).shuffle(buffer_size=1024)
test_data = tf.data.Dataset.from_tensor_slices(X_test).batch(128).shuffle(buffer_size=512)

n_samples = int(len(X_train) + len(X_test))
training_epochs = 20
batch_size = 128
display_step = 1

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
autoencoder.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
autoencoder.summary()

ValueError: ignored

In [16]:
autoencoder.network_weights['recon'][0]['w'].shape

TensorShape([200, 784])

In [0]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)

        # Fit training using batch data
        cost = autoencoder.partial_fit(batch_xs)
        # Compute average loss
        avg_cost += cost / n_samples * batch_size

    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", '%d,' % (epoch + 1),
              "Cost:", "{:.9f}".format(avg_cost))

#print("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

In [0]:

# Iterate over epochs.
for epoch in range(3):
  print('Start of epoch %d' % (epoch,))

  # Iterate over the batches of the dataset.
  for step, x_batch_train in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      reconstructed = vae(x_batch_train)
      # Compute reconstruction loss
      loss = mse_loss_fn(x_batch_train, reconstructed)
      loss += sum(vae.losses)  # Add KLD regularization loss

    grads = tape.gradient(loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(grads, vae.trainable_variables))

    loss_metric(loss)

    if step % 100 == 0:
      print('step %s: mean loss = %s' % (step, loss_metric.result()))